In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import pandas as pd


driver = webdriver.Firefox()

url = "https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats"
driver.get(url)

time.sleep(3)

html = driver.page_source
driver.quit()

soup = BeautifulSoup(html, "html.parser")

table = soup.find("table", id = "results2024-202591_overall")
tbody = table.find("tbody")

club_cells = tbody.find_all('tr')

club_links = []
for tr in club_cells:
    td = tr.find('td', attrs={'data-stat': 'team'})
    club_links.append('https://fbref.com' + td.a['href'])

club_links

['https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats',
 'https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats',
 'https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats',
 'https://fbref.com/en/squads/b2b47a98/2024-2025/Newcastle-United-Stats',
 'https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats',
 'https://fbref.com/en/squads/e4a775cb/2024-2025/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/4ba7cbea/2024-2025/Bournemouth-Stats',
 'https://fbref.com/en/squads/cd051869/2024-2025/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/2024-2025/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/2024-2025/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/d3fd31cc/2024-2025/Everton-Stats',
 'https://fbref.com/en/squads/7c21e445/2024-2025/West-Ham-United-Stats',
 'https://fbref.com/

In [5]:
driver = webdriver.Firefox()

players = []

for link in club_links:
    driver.get(link)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    table = soup.find('table', id='stats_standard_9')
    tbody = table.tbody

    current_player_cells = [tag for tag in tbody.find_all(
        'tr') if tag.get('class') is None]

    h1 = soup.h1.span.get_text()
    club = h1.replace('2024-2025', '').replace('Stats', '').strip()

    for cell in current_player_cells:

        player_dict = {}

        player_dict['player'] = cell.th.a.get_text().strip()
        player_dict['club'] = club

        for td in cell.find_all('td'):
            data_stat = td.get('data-stat')
            if data_stat == 'nationality':
                text = td.get_text()
                value = ''.join([c for c in text if c.isupper()])
            else:
                value = td.get_text().strip() if td.get_text().strip() else ''

            player_dict[data_stat] = value if value!='' else 'N/A'
        
        player_dict.pop('matches')

        minutes_text = player_dict.get('minutes', '0').replace(',', '')
        try:
            playing_time = int(minutes_text)
        except:
            playing_time = 0

        if playing_time > 90:
            player_dict['minutes'] = minutes_text
            players.append(player_dict)

driver.quit()

players

[{'player': 'Mohamed Salah',
  'club': 'Liverpool',
  'nationality': 'EGY',
  'position': 'FW',
  'age': '32',
  'games': '38',
  'games_starts': '38',
  'minutes': '3371',
  'minutes_90s': '37.5',
  'goals': '29',
  'assists': '18',
  'goals_assists': '47',
  'goals_pens': '20',
  'pens_made': '9',
  'pens_att': '9',
  'cards_yellow': '1',
  'cards_red': '0',
  'xg': '25.2',
  'npxg': '18.2',
  'xg_assist': '14.2',
  'npxg_xg_assist': '32.4',
  'progressive_carries': '155',
  'progressive_passes': '144',
  'progressive_passes_received': '488',
  'goals_per90': '0.77',
  'assists_per90': '0.48',
  'goals_assists_per90': '1.25',
  'goals_pens_per90': '0.53',
  'goals_assists_pens_per90': '1.01',
  'xg_per90': '0.67',
  'xg_assist_per90': '0.38',
  'xg_xg_assist_per90': '1.05',
  'npxg_per90': '0.49',
  'npxg_xg_assist_per90': '0.87'},
 {'player': 'Virgil van Dijk',
  'club': 'Liverpool',
  'nationality': 'NED',
  'position': 'DF',
  'age': '33',
  'games': '37',
  'games_starts': '37',


In [6]:

df = pd.DataFrame(players)

df = df.apply(pd.to_numeric, errors='ignore')

type_map = {
    "object": "TEXT",
    "int64": "INTEGER",
    "float64": "REAL"
}

columns_sql = []
for col, dtype in df.dtypes.items():
    sql_type = type_map.get(str(dtype), "TEXT")
    columns_sql.append(f'{col} {sql_type}')

columns_def = ", ".join(columns_sql)

create_table_sql = f'CREATE TABLE IF NOT EXISTS players ({columns_def})'

print(create_table_sql)

conn = sqlite3.connect("players.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS players")

cursor.execute(create_table_sql)
conn.commit()

df.to_sql("players", conn, if_exists='append', index=False)

conn.close()

CREATE TABLE IF NOT EXISTS players (player TEXT, club TEXT, nationality TEXT, position TEXT, age INTEGER, games INTEGER, games_starts INTEGER, minutes INTEGER, minutes_90s REAL, goals INTEGER, assists INTEGER, goals_assists INTEGER, goals_pens INTEGER, pens_made INTEGER, pens_att INTEGER, cards_yellow INTEGER, cards_red INTEGER, xg REAL, npxg REAL, xg_assist REAL, npxg_xg_assist REAL, progressive_carries INTEGER, progressive_passes INTEGER, progressive_passes_received INTEGER, goals_per90 REAL, assists_per90 REAL, goals_assists_per90 REAL, goals_pens_per90 REAL, goals_assists_pens_per90 REAL, xg_per90 REAL, xg_assist_per90 REAL, xg_xg_assist_per90 REAL, npxg_per90 REAL, npxg_xg_assist_per90 REAL)


C:\Users\ducki\AppData\Local\Temp\ipykernel_24148\2628237157.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
